First, load and clean the data. Filter out the " at the end and beginning of each motivation and add a start and end token. Also filter out the <i></i> that occur in some motivations.

In [25]:
import pandas as pd

full_csv = pd.read_csv('data/winner_train.csv')
motivations = full_csv['motivation']
motivations = motivations.dropna()


def prepare_data(data_frame):
    data_frame = data_frame.reset_index(drop=True)
    for i, motivation in enumerate(data_frame):
        # Remove leading and trailing whitespace and double quotes
        motivation = motivation.strip().strip('"')

        # Remove <i> and </i> tags
        motivation = motivation.replace('<i>', '').replace('</i>', '')
        
        # Convert to lowercase and add start/end tokens
        motivation = '#START# ' + motivation.lower() + ' #END#' 
        
        # Update the motivations list with the modified motivation string
        data_frame[i] = motivation

    # Remove duplicates
    data_frame = data_frame.drop_duplicates()
        
    return data_frame


motivations = prepare_data(motivations)


Set up some helper methods

In [26]:
# Count probability of each word
def get_word_count_dict(df):
    word_count = {}
    for row in df:
        for word in row.split():
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
    return word_count


def probability_of_word(word, df):
    word_count = get_word_count_dict(df)
    return word_count[word] / sum(word_count.values())


def bigram_counter(df):
    bigram_count = {}
    for row in df:
        words = row.split()
        for i in range(len(words) - 1):
            bigram = (words[i], words[i+1])
            if bigram in bigram_count:
                bigram_count[bigram] += 1
            else:
                bigram_count[bigram] = 1
    return bigram_count


def sort_dictionary(dictionary):
    sorted_dict = dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=True))
    return sorted_dict


def trigram_counter(df):
    trigram_count = {}
    for row in df:
        words = row.split()
        for i in range(len(words) - 2):
            trigram = (words[i], words[i+1], words[i+2])
            if trigram in trigram_count:
                trigram_count[trigram] += 1
            else:
                trigram_count[trigram] = 1
    return trigram_count


def probability_of_sentence_bigram(sentence, df):
    bigram_count = bigram_counter(df)
    word_count = get_word_count_dict(df)
    words = sentence.split()
    probability = 1

    for i in range(len(words) - 1):
        bigram = (words[i], words[i+1])
        # Divide by number of times the first word appears (and can be followed by another word)
        if bigram in bigram_count:
            to_mult = bigram_count[bigram] / word_count[words[i]]
        else:
            to_mult = 0.00001
            
         # Apply repetition penalty
        for j, word in enumerate(words):
            if (word == words[i] and i !=j):
                to_mult *= 0.000001
     
        probability *= to_mult
    return probability


def probability_of_sentence_trigram(sentence, df):
    trigram_count = trigram_counter(df)
    word_count = get_word_count_dict(df)
    words = sentence.split()
    probability = 1

    for i in range(len(words) - 2):
        trigram = (words[i], words[i+1], words[i+2])
        # Divide by number of times the first word appears (and can be followed by another word)
        if trigram in trigram_count:
            to_mult = trigram_count[trigram] / word_count[words[i]]
        else:
            to_mult = 0.00001
            
         # Apply repetition penalty
        for j, word in enumerate(words):
            if (word == words[i] and i !=j):
                to_mult *= 0.000001
     
        probability *= to_mult    
    return probability



In [27]:
# Version 1: Generation a random motivation
import random
def random_motivation(df):
    bigram_count = bigram_counter(df)
    words = []
    word = '#START#'
    while word != '#END#':
        words.append(word)
        next_word = random.choices(list(bigram_count.keys()), list(bigram_count.values()))
        word = next_word[0][1]
    return ' '.join(words[1:-1])

In [28]:
# Version 2: Generation of the most probable motivation using Bigram 
def most_probable_motivation(df, sorted = True):
    bigram_count = bigram_counter(df)
    if sorted:
        bigram_count = sort_dictionary(bigram_count)
    sentence = ''
    word = '#START#'
    sentence += (word) 
    while word != '#END#' :#and len(sentence.split(' ')) < 10:
        all_bigrams_with_word = [bigram for bigram in bigram_count.keys() if bigram[0] == word]    
        
        if not sorted:
            # take 15 random bigrams from possible_15_words
            length = len(all_bigrams_with_word)
            if length >= 15:
                possible_15_words = random.sample(all_bigrams_with_word, 15)
            else:
                possible_15_words = random.sample(all_bigrams_with_word, length)
        else:
            possible_15_words = all_bigrams_with_word[:15]
        
        if len(possible_15_words) == 0:
            most_common_words = sort_dictionary(dict(islice(
                            get_word_count_dict(motivations).items(), 15)))
            for common in most_common_words:
                possible_15_words.append((word, common))
        
        possible_sentences = [sentence + ' ' + word[1] for word in possible_15_words]
        probabilities = [probability_of_sentence_bigram(sentence, df) for sentence in possible_sentences]

        
        word = possible_15_words[probabilities.index(max(probabilities))][1]
    
        
        sentence += ' ' + (word)
    return sentence


Random motivation chosen from top three 

In [29]:
# Version 3: Generating a probable motivation (introducing randomness), still using bigrams
import numpy as np
from itertools import islice
def most_probable_motivation_with_random(df, sorted = True):
    bigram_count = bigram_counter(df)
    if sorted:
        bigram_count = sort_dictionary(bigram_count)
    sentence = ''
    word = '#START#'
    sentence += (word) 
    while word != '#END#':
        all_bigrams_with_word = [bigram for bigram in bigram_count.keys() if bigram[0] == word]

        if not sorted:
            # take 15 random bigrams from possible_15_words
            length = len(all_bigrams_with_word)
            if length >= 15:
                possible_15_words = random.sample(all_bigrams_with_word, 15)
            else:
                possible_15_words = random.sample(all_bigrams_with_word, length)
        else:
            possible_15_words = all_bigrams_with_word[:15]

        if len(possible_15_words) == 0:
            most_common_words = sort_dictionary(dict(islice(get_word_count_dict(motivations).items(), 15)))
            for common in most_common_words:
                possible_15_words.append((word, common))
        
        
        word_and_prob = {}
        for select in possible_15_words:
            word_and_prob[select[1]] = probability_of_sentence_bigram(sentence + ' ' + select[1], df)

        word_and_prob = sort_dictionary(word_and_prob)
        # Some words don't have three options for their next word
        lenght = len(word_and_prob)
        lenght = (lenght if lenght < 3 else 3) 
        top_three_words = list(word_and_prob.keys())[:lenght]
        word = np.random.choice(top_three_words)
        sentence += ' ' + (word)
    return sentence

In [30]:
# Version 4: Generation of the most probable motivation using trigrams 
def use_bigram_instead(word, sentence, sorted, df):
    bigram_count = bigram_counter(df)
    if sorted:
        bigram_count = sort_dictionary(bigram_count)
    all_bigrams_with_word = [bigram for bigram in bigram_count.keys() if bigram[0] == word]

    if not sorted:
        # take 15 random bigrams from possible_15_words
        length = len(all_bigrams_with_word)
        if length >= 15:
            possible_15_words = random.sample(all_bigrams_with_word, 15)
        else:
            possible_15_words = random.sample(all_bigrams_with_word, length)
    else:
        possible_15_words = all_bigrams_with_word[:15]

    if len(possible_15_words) == 0:
        most_common_words = sort_dictionary(dict(islice(get_word_count_dict(motivations).items(), 15)))
        for common in most_common_words:
            possible_15_words.append((word, common))

    possible_sentences = [sentence + ' ' + word[1] for word in possible_15_words]
    probabilities = [probability_of_sentence_bigram(sentence, df) for sentence in possible_sentences]

    word = possible_15_words[probabilities.index(max(probabilities))][1]
    return word

def most_probable_motivation_trigram(df, sorted = True):
    sorted_val = sorted
    trigram_count = trigram_counter(df)
    if sorted:
        trigram_count = sort_dictionary(trigram_count)
    sentence = ''
    last_word = '#START#'
    sentence += (last_word) 
    while last_word != '#END#' :
        word_to_add = ''

        if len(sentence.split(' ')) == 1 :
            word_to_add = use_bigram_instead(last_word, sentence, sorted_val, df)

        else:
            second_to_last_word = sentence.split(' ')[-2]
            all_trigrams_with_word = [trigram for trigram in trigram_count.keys() if trigram[1] == last_word and trigram[0] == second_to_last_word]
            
            if not sorted:
                length = len(all_trigrams_with_word)
                if length >= 15:
                    possible_15_words = random.sample(all_trigrams_with_word, 15)
                else:
                    possible_15_words = random.sample(all_trigrams_with_word, length)
            else:
                possible_15_words = all_trigrams_with_word[:15]

            if len(possible_15_words) == 0:
                word_to_add = use_bigram_instead(last_word, sentence, sorted_val, df)
                
            else:
                possible_sentences = [sentence + ' ' + word[2] for word in possible_15_words]
                probabilities = [probability_of_sentence_trigram(sentence, df) for sentence in possible_sentences]
                word_to_add = possible_15_words[probabilities.index(max(probabilities))][2]
        
        sentence += ' ' + (word_to_add)
        last_word = word_to_add
    return sentence



Now, the category will also be considered. First set up the data. 

In [31]:
#Create dataframes for the specific categories
df_chemistry = full_csv[full_csv['category'] == 'Chemistry']['motivation']
df_chemistry = df_chemistry.dropna()
df_chemistry = prepare_data(df_chemistry)

df_literature = full_csv[full_csv['category'] == 'Literature']['motivation']
df_literature = df_literature.dropna()
df_literature = prepare_data(df_literature)

df_peace = full_csv[full_csv['category'] == 'Peace']['motivation']
df_peace = df_peace.dropna()
df_peace = prepare_data(df_peace)

df_literature = full_csv[full_csv['category'] == 'Literature']['motivation']
df_literature = df_literature.dropna()
df_literature = prepare_data(df_literature)

df_physics = full_csv[full_csv['category'] == 'Physics']['motivation']
df_physics = df_physics.dropna()
df_physics = prepare_data(df_physics)

df_medicine = full_csv[full_csv['category'] == 'Medicine']['motivation']
df_medicine = df_medicine.dropna()
df_medicine = prepare_data(df_medicine)

df_economics = full_csv[full_csv['category'] == 'Economics']['motivation']
df_economics = df_economics.dropna()
df_economics = prepare_data(df_economics)

df_general = full_csv['motivation']
df_general = df_general.dropna()
df_general = prepare_data(df_general)

Set up helper methods 

In [32]:
#calculate most common words for each category that are not in general
def get_top_15_words(word_count, word_count_general):
    top_15_words = {}
    for word in word_count:
        top_15_words[word] = probability_of_word_in_category(word, word_count, word_count_general)
    top_15_words = sort_dictionary(top_15_words)
    return list(top_15_words.keys())[:15]

def probability_of_word_in_category(word, word_count, word_count_general):
    if word not in word_count:
        return 0.000001
    return max(word_count[word] / word_count_general[word], 0.00001)

In [33]:
#Generate most probable motivation for each category using bigrams
def most_probable_motivation_for_category(df, df_category, sorted = True):
    word_count_general = get_word_count_dict(df)
    word_count_cat = get_word_count_dict(df_category)
    common_words_for_cat = get_top_15_words(word_count_cat, word_count_general)  

    bigram_count = bigram_counter(df)
    if sorted:
        bigram_count = sort_dictionary(bigram_count)
    sentence = ''
    word = '#START#'
    sentence += (word) 
    while word != '#END#' :
        possible_words = []
        all_bigrams_with_word = [bigram for bigram in bigram_count.keys() if bigram[0] == word]    
        
        if not sorted:
            length = len(all_bigrams_with_word)
            if length >= 15:
                possible_15_words = random.sample(all_bigrams_with_word, 15)
            else:
                possible_15_words = random.sample(all_bigrams_with_word, length)
        else:
            possible_15_words = all_bigrams_with_word[:15]
        possible_words = possible_15_words
   
        #Add the most common words for the category to the possible words
        for word in common_words_for_cat:      
            possible_words.append((possible_15_words[0][0], word))

        #create 30 possible sentences
        possible_sentences = [sentence + ' ' + bigram[1] for bigram in possible_words]
       
        probabilities = []
        for sent in possible_sentences:
            prob = probability_of_sentence_bigram(sent, df) * probability_of_word_in_category(sent.split()[-1], word_count_cat, word_count_general)
            probabilities.append(prob)
        
        word = possible_words[probabilities.index(max(probabilities))][1]
       
        sentence += ' ' + (word)
    return sentence




Evaluation, first using the BLEU score

In [34]:
from nltk.translate.bleu_score import sentence_bleu

Helper method for evaluation

In [35]:
def remove_first_last_word(string):
    words = string.split()
    if len(words) > 2:
        return ' '.join(words[1:-1])
    else:
        return ''


In [36]:
# Take max, min and median/mean of the scores
# All existing motivations each in list format
eval_csv = pd.read_csv('data/winner_eval.csv')
#choose column named motivation
eval = eval_csv['motivation']
eval = prepare_data(eval)
motivation_split_list = [sentence.split()[1:-1] for sentence in eval]


# Evaluating random motivation
scores_random = []
for i in range(10):
    all_scores = []
    candidate = random_motivation(motivations)
    for i in range(len(motivation_split_list)):
        score = sentence_bleu(motivation_split_list[i], candidate, weights=(0.25, 0.25, 0.25, 0.25))
        scores_random.append(score)
    all_scores.append(np.mean(scores_random))
scores_random.sort()


# Evaluating most probable motivation, bigram, sorted
scores_most_probable_bigram_sort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation(motivations, sorted = True)
    for i in range(len(motivation_split_list)):
        score = sentence_bleu(motivation_split_list[i], remove_first_last_word(candidate), weights=(0.25, 0.25, 0.25, 0.25))
        scores_most_probable_bigram_sort.append(score)
    all_scores.append(np.mean(scores_most_probable_bigram_sort))
scores_most_probable_bigram_sort.sort()

# Evaluating most probable motivation, bigram, unsorted
scores_most_probable_bigram_NOsort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation(motivations, sorted = False)
    for i in range(len(motivation_split_list)):
        score = sentence_bleu(motivation_split_list[i], remove_first_last_word(candidate), weights=(0.25, 0.25, 0.25, 0.25))
        scores_most_probable_bigram_NOsort.append(score)
    all_scores.append(np.mean(scores_most_probable_bigram_NOsort))
scores_most_probable_bigram_NOsort.sort()


# Evaluating most probable motivation with randomness, sorted
scores_most_probable_random_sort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation_with_random(motivations, sorted = True)
    for i in range(len(motivation_split_list)):
        score = sentence_bleu(motivation_split_list[i], remove_first_last_word(candidate), weights=(0.25, 0.25, 0.25, 0.25))
        scores_most_probable_random_sort.append(score)
    all_scores.append(np.mean(scores_most_probable_random_sort))
scores_most_probable_random_sort.sort()



# Evaluating most probable motivation with randomness, unsorted
scores_most_probable_random_NOsort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation_with_random(motivations, sorted = False)
    for i in range(len(motivation_split_list)):
        score = sentence_bleu(motivation_split_list[i], remove_first_last_word(candidate), weights=(0.25, 0.25, 0.25, 0.25))
        scores_most_probable_random_NOsort.append(score)
    all_scores.append(np.mean(scores_most_probable_random_NOsort))
scores_most_probable_random_NOsort.sort()


# Evaluating most probable motivation with trigram, sorted
scores_most_probable_trigram_sort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation_trigram(motivations, sorted = True)
    for i in range(len(motivation_split_list)):
        score = sentence_bleu(motivation_split_list[i], remove_first_last_word(candidate), weights=(0.25, 0.25, 0.25, 0.25))
        scores_most_probable_trigram_sort.append(score)
    all_scores.append(np.mean(scores_most_probable_trigram_sort))
scores_most_probable_trigram_sort.sort()

# Evaluating most probable motivation with trigram, unsorted
scores_most_probable_trigram_NOsort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation_trigram(motivations, sorted = False)
    for i in range(len(motivation_split_list)):
        score = sentence_bleu(motivation_split_list[i], remove_first_last_word(candidate), weights=(0.25, 0.25, 0.25, 0.25))
        scores_most_probable_trigram_NOsort.append(score)
    all_scores.append(np.mean(scores_most_probable_trigram_NOsort))
scores_most_probable_trigram_NOsort.sort()


scores_category = []
category_dfs = [df_chemistry, df_literature, df_peace, df_physics, df_medicine, df_economics]
for df in category_dfs:
    all_scores = []
    for i in range(2):
        candidate = most_probable_motivation_for_category(motivations, df, sorted = False)
        for i in range(len(motivation_split_list)):
            score = sentence_bleu(motivation_split_list[i], remove_first_last_word(candidate), weights=(0.25, 0.25, 0.25, 0.25))
            scores_category.append(score)      
scores_category.sort()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [37]:
# Ranom generation
print("*Random result*", "\n", "mean =", np.mean(scores_random),"\n","Median =", np.median(scores_random),"\n","min =", scores_random[0], "\n", "max =", scores_random[-1], "\n")

# Most probable deterministic
print("*Most Probable result (Bigram, sorted)*", "\n", "mean =", np.mean(scores_most_probable_bigram_sort),"\n","Median =", np.median(scores_most_probable_bigram_sort),"\n","min =", scores_most_probable_bigram_sort[0], "\n", "max =", scores_most_probable_bigram_sort[-1], "\n")

# Most probable deterministic not sorted
print("*Most Probable result (Bigram, unsorted)*", "\n", "mean =", np.mean(scores_most_probable_bigram_NOsort),"\n","Median =", np.median(scores_most_probable_bigram_NOsort),"\n","min =", scores_most_probable_bigram_NOsort[0], "\n", "max =", scores_most_probable_bigram_NOsort[-1], "\n")

# Most probable with randomness
print("*Most Probable with Random result (Bigram, sorted)*", "\n", "mean =", np.mean(scores_most_probable_random_sort),"\n","Median =", np.median(scores_most_probable_random_sort),"\n","min =", scores_most_probable_random_sort[0], "\n", "max =", scores_most_probable_random_sort[-1], "\n")

# Most probable with randomness not sorted
print("*Most Probable with Random result (Bigram, unsorted)*", "\n", "mean =", np.mean(scores_most_probable_random_NOsort),"\n","Median =", np.median(scores_most_probable_random_NOsort),"\n","min =", scores_most_probable_random_NOsort[0], "\n", "max =", scores_most_probable_random_NOsort[-1], "\n")

# Most probable with trigram
print("*Most Probable result (Trigram, sorted)*", "\n", "mean =", np.mean(scores_most_probable_trigram_sort),"\n","Median =", np.median(scores_most_probable_trigram_sort),"\n","min =", scores_most_probable_trigram_sort[0], "\n", "max =", scores_most_probable_trigram_sort[-1], "\n")

# Most probable with trigram not sorted
print("*Most Probable result (Trigram, unsorted)*", "\n", "mean =", np.mean(scores_most_probable_trigram_NOsort),"\n","Median =", np.median(scores_most_probable_trigram_NOsort),"\n","min =", scores_most_probable_trigram_NOsort[0], "\n", "max =", scores_most_probable_trigram_NOsort[-1], "\n")

# Most probable with category
print("*Most Probable result (Category and bigram, unsorted)*", "\n", "mean =", np.mean(scores_category),"\n","Median =", np.median(scores_category),"\n","min =", scores_category[0], "\n", "max =", scores_category[-1], "\n")

*Random result* 
 mean = 0.0919158751848741 
 Median = 0.08163579932055849 
 min = 0 
 max = 0.3244913214932798 

*Most Probable result (Bigram, sorted)* 
 mean = 0.19056844977054535 
 Median = 0.16851375741211452 
 min = 3.184491103071845e-78 
 max = 0.40611607179711245 

*Most Probable result (Bigram, unsorted)* 
 mean = 0.11679320360436268 
 Median = 0.10969605945035435 
 min = 1.6227569201872753e-78 
 max = 0.3484162270015532 

*Most Probable with Random result (Bigram, sorted)* 
 mean = 0.1354741903405869 
 Median = 0.10624821699408675 
 min = 8.759666304185185e-155 
 max = 0.6788877661016413 

*Most Probable with Random result (Bigram, unsorted)* 
 mean = 0.0598208544811542 
 Median = 0.04194807103827805 
 min = 7.310577634934788e-155 
 max = 0.31837754460503936 

*Most Probable result (Trigram, sorted)* 
 mean = 0.056006595919949324 
 Median = 0.05789421994462952 
 min = 1.7159929819832364e-78 
 max = 0.09979654358436284 

*Most Probable result (Trigram, unsorted)* 
 mean = 0.10

Now, evaluation using semantic similarity

In [38]:
!pip install sentence-transformers torch scipy

In [39]:
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cosine

# Use a sentence transformer since the motivations has to be embedded to use cosine similarity
model_name = "paraphrase-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

def semantic_similarity_score(sentence1, sentence2):
    embeddings1 = model.encode(sentence1, convert_to_tensor=True)
    embeddings2 = model.encode(sentence2, convert_to_tensor=True)
    score = 1 - cosine(embeddings1, embeddings2)
    return score


ModuleNotFoundError: No module named 'sentence_transformers'

In [ ]:
eval_csv = pd.read_csv('winner_eval.csv')
eval = eval_csv['motivation']
eval = prepare_data(eval)
motivations_list = eval[1:]

# Evaluating random motivation
scores_random = []
for i in range(10):
    all_scores = []
    candidate = random_motivation(motivations)
    for sentence in motivations_list:
        score = semantic_similarity_score(remove_first_last_word(sentence), candidate)
        scores_random.append(score)
    all_scores.append(np.mean(scores_random))
scores_random.sort()


# Evaluating most probable motivation, bigram, sorted
scores_most_probable_bigram_sort = []
for i in range(1):
    all_scores = []
    candidate = most_probable_motivation(motivations, sorted = True)
    for sentence in motivations_list:
        score = semantic_similarity_score(remove_first_last_word(sentence), remove_first_last_word(candidate))
        scores_most_probable_bigram_sort.append(score)
    all_scores.append(np.mean(scores_most_probable_bigram_sort))
scores_most_probable_bigram_sort.sort()


# Evaluating most probable motivation, bigram, unsorted
scores_most_probable_bigram_NOsort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation(motivations, sorted = False)
    for sentence in motivations_list:
        score = semantic_similarity_score(remove_first_last_word(sentence), remove_first_last_word(candidate))
        scores_most_probable_bigram_NOsort.append(score)
    all_scores.append(np.mean(scores_most_probable_bigram_NOsort))
scores_most_probable_bigram_NOsort.sort()


# Evaluating most probable motivation with randomness, sorted
scores_most_probable_random_sort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation_with_random(motivations, sorted = True)
    for sentence in motivations_list:
        score = semantic_similarity_score(remove_first_last_word(sentence), remove_first_last_word(candidate))
        scores_most_probable_random_sort.append(score)
    all_scores.append(np.mean(scores_most_probable_random_sort))
scores_most_probable_random_sort.sort()


# Evaluating most probable motivation with randomness, unsorted
scores_most_probable_random_NOsort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation_with_random(motivations, sorted = False)
    for sentence in motivations_list:
        score = semantic_similarity_score(remove_first_last_word(sentence), remove_first_last_word(candidate))
        scores_most_probable_random_NOsort.append(score)
    all_scores.append(np.mean(scores_most_probable_random_NOsort))
scores_most_probable_random_NOsort.sort()


# Evaluating most probable motivation with trigram, sorted
scores_most_probable_trigram_sort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation_trigram(motivations, sorted = True)
    for sentence in motivations_list:
        score = semantic_similarity_score(remove_first_last_word(sentence), remove_first_last_word(candidate))
        scores_most_probable_trigram_sort.append(score)
    all_scores.append(np.mean(scores_most_probable_trigram_sort))
scores_most_probable_trigram_sort.sort()


# Evaluating most probable motivation with trigram, unsorted
scores_most_probable_trigram_NOsort = []
for i in range(10):
    all_scores = []
    candidate = most_probable_motivation_trigram(motivations, sorted = False)
    for sentence in motivations_list:
        score = semantic_similarity_score(remove_first_last_word(sentence), remove_first_last_word(candidate))
        scores_most_probable_trigram_NOsort.append(score)
    all_scores.append(np.mean(scores_most_probable_trigram_NOsort))
scores_most_probable_trigram_NOsort.sort()

# Evaluating most probable motivation with category
scores_category = []
category_dfs = [df_chemistry, df_literature, df_peace, df_physics, df_medicine, df_economics]
for df in category_dfs:
    all_scores = []
    for i in range(2):
        candidate = most_probable_motivation_for_category(motivations, df, sorted = False)
        for sentence in motivations_list:
            score = semantic_similarity_score(remove_first_last_word(sentence), remove_first_last_word(candidate))
            scores_category.append(score)
scores_category.sort()

In [ ]:
# Ranom generation
print("*Random result*", "\n", "mean =", np.mean(scores_random),"\n","Median =", np.median(scores_random),"\n","min =", scores_random[0], "\n", "max =", scores_random[-1], "\n")

# Most probable deterministic
print("*Most Probable result (Bigram, sorted)*", "\n", "mean =", np.mean(scores_most_probable_bigram_sort),"\n","Median =", np.median(scores_most_probable_bigram_sort),"\n","min =", scores_most_probable_bigram_sort[0], "\n", "max =", scores_most_probable_bigram_sort[-1], "\n")

# Most probable deterministic not sorted
print("*Most Probable result (Bigram, unsorted)*", "\n", "mean =", np.mean(scores_most_probable_bigram_NOsort),"\n","Median =", np.median(scores_most_probable_bigram_NOsort),"\n","min =", scores_most_probable_bigram_NOsort[0], "\n", "max =", scores_most_probable_bigram_NOsort[-1], "\n")

# Most probable with randomness
print("*Most Probable with Random result (Bigram, sorted)*", "\n", "mean =", np.mean(scores_most_probable_random_sort),"\n","Median =", np.median(scores_most_probable_random_sort),"\n","min =", scores_most_probable_random_sort[0], "\n", "max =", scores_most_probable_random_sort[-1], "\n")

# Most probable with randomness not sorted
print("*Most Probable with Random result (Bigram, unsorted)*", "\n", "mean =", np.mean(scores_most_probable_random_NOsort),"\n","Median =", np.median(scores_most_probable_random_NOsort),"\n","min =", scores_most_probable_random_NOsort[0], "\n", "max =", scores_most_probable_random_NOsort[-1], "\n")

# Most probable with trigram
print("*Most Probable result (Trigram, sorted)*", "\n", "mean =", np.mean(scores_most_probable_trigram_sort),"\n","Median =", np.median(scores_most_probable_trigram_sort),"\n","min =", scores_most_probable_trigram_sort[0], "\n", "max =", scores_most_probable_trigram_sort[-1], "\n")

# Most probable with trigram not sorted
print("*Most Probable result (Trigram, unsorted)*", "\n", "mean =", np.mean(scores_most_probable_trigram_NOsort),"\n","Median =", np.median(scores_most_probable_trigram_NOsort),"\n","min =", scores_most_probable_trigram_NOsort[0], "\n", "max =", scores_most_probable_trigram_NOsort[-1], "\n")

# Most probable with category
print("*Most Probable result (Category and bigram, unsorted)*", "\n", "mean =", np.mean(scores_category),"\n","Median =", np.median(scores_category),"\n","min =", scores_category[0], "\n", "max =", scores_category[-1], "\n")

*Random result* 
 mean = 0.18020729031413793 
 Median = 0.17391572892665863 
 min = -0.034190766513347626 
 max = 0.44707831740379333 

*Most Probable result (Bigram, sorted)* 
 mean = 0.30634115636348724 
 Median = 0.3047269582748413 
 min = 0.18095824122428894 
 max = 0.45845159888267517 

*Most Probable result (Bigram, unsorted)* 
 mean = 0.19249040284194052 
 Median = 0.18608178943395615 
 min = -0.009160984307527542 
 max = 0.4328368008136749 

*Most Probable with Random result (Bigram, sorted)* 
 mean = 0.21444718047976494 
 Median = 0.2089853212237358 
 min = 0.01768127828836441 
 max = 0.4470449388027191 

*Most Probable with Random result (Bigram, unsorted)* 
 mean = 0.20356779375113546 
 Median = 0.19679010659456253 
 min = -0.04162440448999405 
 max = 0.5702179074287415 

*Most Probable result (Trigram, sorted)* 
 mean = 0.20026683062314987 
 Median = 0.1787073016166687 
 min = 0.0983252003788948 
 max = 0.3128131628036499 

*Most Probable result (Trigram, unsorted)* 
 mean 